## HW 10

In [1]:
#Imports

import statsmodels.tsa.api as tsa
from statsmodels.tsa.tsatools import unvec, vec
import numpy as np
import scipy as sp
from scipy.linalg import toeplitz

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

### Part 2

#### a) Find a state space representation for $z_t = \Delta y_t$ with a stationary state vector $x_t$. Use the notation [here](https://niskrev.github.io/PhDEcon108/01-Lectures/07-Multivariate/02-StateSpace.html). Use $d$ for the intercept in the observation equation (This term is missing in the representation in the link, you can think of $z$ being de-meaned: $z_t = \bar{z}_t - d$)

## UC model

With

$$z_t =  \Delta y_t = \tau_t - \tau_{t-1} + c_t - c_{t-1} = d + \eta_t + \Delta c_t $$


State space form is: 

$$ 
\begin{align}
X = \left[\begin{array}{c}
c_{t}\\
c_{t-1}\\
\end{array}\right]
 &=
\begin{bmatrix}
         \phi_1 & \phi_2 \\
         1 & 0
    \end{bmatrix}
\left[\begin{array}{c}
c_{t-1}\\
c_{t-2}\\
\end{array}\right] 
+
\left[\begin{array}{c}
1\\
0\\
\end{array}\right]\varepsilon_{t} , \;\;\;\;\; \varepsilon_{t} \sim \mathcal{N} \left( 0, \;\mathbf{\Sigma}_{\varepsilon}\right)
\\\\
z_t & = \left[\begin{array}{cc} 1 \; -1 \end{array} \right] 
X  + d + \eta_t \\\\
\end{align}
$$

#### b)  Write a python function which takes values for the model parameters as input and returns the matrices of the state space representation as output. 

In [2]:
def staterep(d, phi1, phi2, sigma_eta, sigma_eps):

    design = np.array([[1., -1.]])
    obs_intercept = np.array([[d]])
    transition = np.array([[phi1, phi2],[1., 0]])
    selection = np.array([[1.],[0]])
    obs_cov = np.array([[sigma_eta ** 2]])
    state_cov = np.array([[sigma_eps ** 2]])
    
    return design, obs_intercept, transition, selection, obs_cov, state_cov

In [3]:
#Values from Morley, Nelson & Zivot (2003, p.238, Table 1)
d=0.8119

[design, obs_intercept, transition, selection, obs_cov, state_cov] = staterep(d, 1.5303, -0.6097, 0.6893, 0.6199)

#### c) Construct the covariance matrix of the marginal distribution of $X$ for T=2

Marginal distribution of $\boldsymbol X = [\mathbf{x}^{\prime}_{1}, \mathbf{x}^{\prime}_{2}]^{\prime}$

$$
\underset{\boldsymbol X}{\underbrace{
\left[\begin{array}{c}
\mathbf{x}_{1}\\
\mathbf{x}_{2}\\
\end{array}\right]}}
=
\underset{\boldsymbol A}{\underbrace{
\left[\begin{array}{cccccccc}
A & I & 0 \\
A^2 & A & I
\end{array}\right]}}
\underset{\boldsymbol E}{\underbrace{
\left[\begin{array}{c}
\mathbf{x}_{0}\\
\boldsymbol \varepsilon_{1}\\
\boldsymbol \varepsilon_{2}
\end{array}\right]}}
$$


$$
\boldsymbol X \sim \mathcal{N} \left( 0, \; \boldsymbol A \mathbf{\Sigma}_{\boldsymbol E} \boldsymbol A^{\prime}\right)
$$


In [4]:
#building the covariance matrix of the marginal distribution of X
SigE = selection@state_cov@selection.T

vecx0=np.linalg.solve(np.eye((2*1)**2)-np.kron(transition, transition), vec(SigE))
x0=unvec(vecx0)

bigA = np.concatenate((np.concatenate((np.concatenate((transition, transition@transition)),np.concatenate((np.identity(2),transition))),axis=1),np.concatenate((np.zeros((2,2)),np.identity(2)))),axis=1)

SigBigE=sp.linalg.block_diag(x0,SigE,SigE)

covX=bigA@SigBigE@bigA.T
covX

array([[6.35681416, 6.04325819, 6.04325819, 6.35681416],
       [6.04325819, 6.35681416, 5.37224841, 6.04325819],
       [6.04325819, 5.37224841, 6.35681416, 6.04325819],
       [6.35681416, 6.04325819, 6.04325819, 6.35681416]])

#### d) Construct the mean vector and covariance matrix of the marginal distribution of $Z$ for T=2 

#### Marginal distribution of $\boldsymbol Z = \left[\mathbf{z}^{\prime}_{1}, \mathbf{z}^{\prime}_{2} \right]^{\prime}$

$$
\underset{\boldsymbol Z}{\underbrace{
\left[\begin{array}{c}
\mathbf{z}_{1}\\
\mathbf{z}_{2}
\end{array}\right]}}
=
\underset{\boldsymbol C}{\underbrace{
\left[\begin{array}{cccc}
C &  0 \\
0 &  C \\
\end{array}\right]}}
\underset{\boldsymbol X}{\underbrace{
\left[\begin{array}{c}
\mathbf{x}_{1}\\
\mathbf{x}_{2}
\end{array}\right]}}
+
\underset{\boldsymbol V}{\underbrace{
\left[\begin{array}{c}
\boldsymbol \eta_{1}\\
\boldsymbol \eta_{2}
\end{array}\right]}}
+ d
$$

In [5]:
#building the covariance matrix of the marginal distribution of Z
bigC = sp.linalg.block_diag(design,design)

SigV = obs_cov
SigBigV = sp.linalg.block_diag(SigV,SigV)

covZ = bigC@covX@bigC.T + SigBigV
covZ

array([[1.10224643, 0.35745381],
       [0.35745381, 1.10224643]])

In [6]:
#building the mean vector
meanZ = np.array([[d],[d]])
meanZ

array([[0.8119],
       [0.8119]])

#### e) Write a function which takes model parameter values and a sample size T as inputs and returns the mean vector and covariance matrix of the marginal distribution of $Z$.

In [7]:
def magdistZ(d, phi1, phi2, sigma_eta, sigma_eps, T):
    
    design = np.array([[1., -1.]])
    obs_intercept = np.array([[d]])
    transition = np.array([[phi1, phi2],[1., 0]])
    selection = np.array([[1.],[0]])
    obs_cov = np.array([[sigma_eta ** 2]])
    state_cov = np.array([[sigma_eps ** 2]])
    
    SigE = selection@state_cov@selection.T

    vecx0=np.linalg.solve(np.eye((2*1)**2)-np.kron(transition, transition), vec(SigE))
    x0=unvec(vecx0)

    for t in range(0,T):
        A = np.zeros(T)
        A[T] = np.linalg.matrix_power(transition, T)
        r = np.zeros(T)
        bigA = toeplitz(A, np.identity(2), r)

    SigBigE=sp.linalg.block_diag(x0,SigE,SigE)
    covX=bigA@SigBigE@bigA.T
    
    aux=design.reshape(1,1,2).repeat(T,axis=0)
    bigC=sp.linalg.block_diag( *aux )
    
    SigV = obs_cov
    SigBigV = np.identity(T)*SigV
    
    covZ = bigC@covX@bigC.T + SigBigV
    meanZ = np.full((T, 1), d)
    
    return covZ, meanZ

In [8]:
#teste
magdistZ(d, 1.5303, -0.6097, 0.6893, 0.6199, 100)

IndexError: index 100 is out of bounds for axis 0 with size 100

#### f) Draw a sample from the distribution of $Z$ for T=100 and compute the log-likelihood of $Z$.